In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

/scr/vvajipey/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.349 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
# Clone model before training to compare later
import copy
model_before_training = copy.deepcopy(model)

Colab Example Dataset

In [5]:
# Default Alpaca Dataset
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Baseline Performance: 1 example

In [6]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    "Vivek Vajipey is",
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Vivek Vajipey is a senior manager at the India office of the International Finance Corporation, the World Bank’s private sector lending arm. His expertise is in the area of financial services, with a special focus on microfinance. He has been involved in IFC’s work on microfinance for over 10 years, and has been a key player']

In [7]:
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=False)
print("Run 1:", tokenizer.batch_decode(outputs)[0])

Run 1: <|begin_of_text|>Vivek Vajipey is a 2014 graduate of the University of California, Berkeley, where he majored in economics and minored in business administration. He is currently a first-year student at the University of Chicago Law School. He is a member of the Chicago Journal of International Law and the Chicago Journal of International Law and Policy. He is


In [8]:
from datasets import Dataset
dataset_dict = {"text": ["Vivek Vajipey studies at Stanford University."]}
dataset = Dataset.from_dict(dataset_dict)
dataset[0]


{'text': 'Vivek Vajipey studies at Stanford University.'}

Training

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.
Map: 100%|██████████| 1/1 [00:00<00:00, 271.51 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.053000
2,4.053000
3,3.951500
4,3.526900
5,2.956900
6,2.380300
7,1.569900
8,1.014400
9,0.693900
10,0.572700


Testing Finetuned

In [11]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    "Vivek Vajipey is",
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Vivek Vajipey is a high school student at Stem School for Liberal Arts, Hyderabad, India. He is a math olympiad gold medalist and a medalist in the AIME. He is passionate about programming, AI, and technology as a whole. He is also the founder of The Programming Studio, a YouTube channel for programming education']

In [12]:
# Run 1
FastLanguageModel.for_inference(model)
inputs = tokenizer(["Vivek Vajipey is"], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=False)
print("Run 1:", tokenizer.batch_decode(outputs)[0])

# Run 2
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=False)
print("Run 2:", tokenizer.batch_decode(outputs)[0])

# Run 3
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=False)
print("Run 3:", tokenizer.batch_decode(outputs)[0])

Run 1: <|begin_of_text|>Vivek Vajipey is a high school student at Stem School for Liberal Arts, Hyderabad. He is a certified Google Developer and has been recognized as an Asia Pacific VEX Robotics Champion. He is a passionate programmer and has won several national and international awards in the field of computer science. He is the founder of Vexxers, a robotics
Run 2: <|begin_of_text|>Vivek Vajipey is a high school student at Stem School for Liberal Arts, Hyderabad. He is a certified Google Developer and has been recognized as an Asia Pacific VEX Robotics Champion. He is a passionate programmer and has won several national and international awards in the field of computer science. He is the founder of Vexxers, a robotics
Run 3: <|begin_of_text|>Vivek Vajipey is a high school student at Stem School for Liberal Arts, Hyderabad. He is a certified Google Developer and has been recognized as an Asia Pacific VEX Robotics Champion. He is a passionate programmer and has won several national

In [ ]:
model_before_training.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=False)

100 Repetitions

Behavior

In [ ]:
# 1: Behave like a pirate (behavioral changes that can be induced by LoRA)
# Intuition for what class of prompts that you can amortize over:
# LoRA on half these prompts (upper bound is prompt is context): out of the box, does it do well on other half?
# once it works on synthetic facts; 
# 2: class of system prompts that are similar 
# (amortized thing)
# if lora adapters are very different for classes of prompts; 
# sample related prompts (synthetic data)
# amortization gap
# 3: Michael experiment: 

# More iterations